In [1]:
!pip install colpali-engine>=0.3.4
!pip install -U bitsandbytes -q
!pip install mteb transformers tqdm typer seaborn -q
!pip install pdf2image einops google-generativeai gradio -q
!apt-get install poppler-utils
!pip install cloudinary
import pprint
from typing import List, cast

import torch
from datasets import Dataset, load_dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
from pdf2image import convert_from_path


from colpali_engine.models import ColPali
from colpali_engine.models.paligemma.colpali.processing_colpali import ColPaliProcessor
from colpali_engine.utils.processing_utils import BaseVisualRetrieverProcessor
from colpali_engine.utils.torch_utils import ListDataset, get_torch_device


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.6.1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 15.0 MB/s eta 0:00:00
   ━━━━

In [2]:
device = get_torch_device("auto")
print(f"Device used: {device}")

    # Model name
model_name = "vidore/colpali-v1.2"

    # Load model
model = ColPali.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        device_map=device,
    ).eval()
model.save_pretrained("/content/", safe_serialization=True)  # Aggiungi questa opzione per safetensors
    # Load processor
processor = cast(ColPaliProcessor, ColPaliProcessor.from_pretrained(model_name))

Device used: cuda:0


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/750 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/66.3k [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/integrations/peft.py:418: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/243k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/733 [00:00<?, ?B/s]

In [3]:
images = []
document_embeddings = []
files = ['/content/Cartella_Clinica_Paziente1.pdf','/content/Cartella_Clinica_Paziente2.pdf']
for file in files:
        print(f"Indexing now: {file}")
        images.extend(convert_from_path(file))



    # Create DataLoader for image batches
dataloader = DataLoader(
        images,
        batch_size=1,
        shuffle=False,
        collate_fn=lambda x: processor.process_images(x),
    )

    # Process each batch and obtain embeddings
for batch in dataloader:
        with torch.no_grad():
            batch = {key: value.to(device) for key, value in batch.items()}
            embeddings = model(**batch)
        document_embeddings.extend(list(torch.unbind(embeddings.to("cpu"))))
total_memory = sum(embedding.element_size() * embedding.nelement() for embedding in document_embeddings)
print(f'Total Embedding Memory (CPU): {total_memory/1024 **2} MB')

Indexing now: /content/Cartella_Clinica_Paziente1.pdf
Indexing now: /content/Cartella_Clinica_Paziente2.pdf
Total Embedding Memory (CPU): 1.005859375 MB


In [34]:
!pip install jsonlines
import jsonlines
import json
import requests
import base64
from io import BytesIO
def generate_content_llama(query, image):
  url = "https://api.aimlapi.com/chat/completions"
  # Lista per salvare le immagini in formato base64
  buffered = BytesIO()
  image.save(buffered, format="PNG")  # Puoi scegliere il formato desiderato (PNG, JPEG, ecc.)
  img_base64 = base64.b64encode(buffered.getvalue()).decode("utf-8")

  import cloudinary
  import cloudinary.uploader
  import cloudinary.api

# Configurazione di Cloudinary con la tua API Key, API Secret e Cloud Name
  cloudinary.config(
    cloud_name = 'CLOUD NAME',  # Sostituisci con il tuo Cloud Name
    api_key = 'API KEY',        # Sostituisci con la tua API Key
    api_secret = 'API SECRET'   # Sostituisci con il tuo API Secret
  )

# Il percorso del file immagine che desideri caricare
  response = cloudinary.uploader.upload("data:image/png;base64," + img_base64)
  print(response['url'])
  payload = json.dumps({
    "model": "meta-llama/Llama-3.2-11B-Vision-Instruct-Turbo",
    "messages": [
      {
        "role": "user",
        "content": [
          {"type": "text", "text": "You are an chatbot assistant for doctors, be precise in what you say according to the image provided, about this question: "+query},
          {
          "type": "image_url",
          "image_url": {
            "url": response['url']
          }
        }
        ]
      }
    ],
    "max_tokens": 300
  })

  headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer APIKEY'
  }

  response = requests.post(url, headers=headers, data=payload)
  print(response.json())
  content = response.json()['choices'][0]['message']['content']
  print(content)
  return content

In [6]:
!pip install transformers torch
from transformers import LlamaForCausalLM, LlamaTokenizer
import torch
def answer_to_question(question, model):

    queries = [question]
    print(question)
    dataloader = DataLoader(
        queries,
        batch_size=1,
        shuffle=False,
        collate_fn=lambda x: processor.process_queries(x),
    )
    qs = []
    for batch_query in dataloader:
            with torch.no_grad():
                batch_query = {k: v.to(model.device) for k, v in batch_query.items()}
                embeddings_query = model(**batch_query)
            qs.extend(list(torch.unbind(embeddings_query.to("cpu"))))

    scores = processor.score(qs, document_embeddings).cpu().numpy()
    best_indexes = scores.argmax(axis=1)
    print(best_indexes)
    count=0
    for x in best_indexes:
        response = generate_content_llama(queries[0], images[best_indexes[0]])

    return response

In [32]:

a= answer_to_question("is Mr Rossi a Smoker?", model)
print(a)

Il Signor Rossi è un fumatore?
[0]
http://res.cloudinary.com/dtul1m2a5/image/upload/v1733045037/bcnr5rxps2sfaysjvueg.png
{'id': '8eb205836d008e5d-PDX', 'object': 'chat.completion', 'choices': [{'index': 0, 'finish_reason': 'stop', 'logprobs': None, 'message': {'role': 'assistant', 'content': 'Sì, il Signor Rossi è un fumatore, come indicato nella sua scheda clinica. La scheda conferma che il Signor Rossi fuma 20 sigarette al giorno, per 15 anni, con una cessazione di 5 anni fa.', 'tool_calls': []}}], 'created': 1733045039, 'model': 'meta-llama/Llama-3.2-11B-Vision-Instruct-Turbo', 'usage': {'prompt_tokens': 8, 'completion_tokens': 22, 'total_tokens': 30}}
Sì, il Signor Rossi è un fumatore, come indicato nella sua scheda clinica. La scheda conferma che il Signor Rossi fuma 20 sigarette al giorno, per 15 anni, con una cessazione di 5 anni fa.
Sì, il Signor Rossi è un fumatore, come indicato nella sua scheda clinica. La scheda conferma che il Signor Rossi fuma 20 sigarette al giorno, per 

In [ ]:
!pip install fastapi uvicorn nest-asyncio pyngrok

from fastapi import FastAPI
import nest_asyncio
from pyngrok import ngrok
import uvicorn
#!ngrok config add-authtoken TOKEN
# Permetti al loop asyncio di funzionare su Colab
nest_asyncio.apply()

# Crea l'app FastAPI
app = FastAPI()

# Definisci un endpoint semplice
@app.get("/")
def read_root(param1: str):
  a=answer_to_question(param1, model)
  return {"message": a}

@app.get("/items/{item_id}")
def read_item(item_id: int, q: str = None):
    return {"item_id": item_id, "q": q}

# Avvia il server
def run_app():
    # Avvia un tunnel pubblico con ngrok
    public_url = ngrok.connect(8000).public_url
    print(f"FastAPI è disponibile pubblicamente su: {public_url}")

    # Esegui Uvicorn per servire l'app
    uvicorn.run(app, host="0.0.0.0", port=8000)

run_app()

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
FastAPI è disponibile pubblicamente su: https://ee14-34-125-104-108.ngrok-free.app


INFO:     Started server process [184]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     82.50.38.130:0 - "GET / HTTP/1.1" 422 Unprocessable Entity
